In [1]:
import sys 
sys.path.append('..')

In [2]:
import pandas as pd
from os import listdir
from os.path import isfile, join
from utils.functions import dropna_pearsonr, dropna_spearmanr, get_df_corr
from climate.dendroclim import plot_mothly_dendroclim
from climate.coh import get_coh_corr_table
import numpy as np

pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
path = '../input/climate/real+grid'
files = [f.split('.')[0] for f in listdir(path) if isfile(join(path, f))]

data = dict()
for f in files:
    data[f] = pd.read_csv(f'{path}/{f}.csv')

data['DP_Inuvik'] = pd.read_csv('../input/climate/real/dew point temperature/DewPointTemp_Inuvik.csv')
data['SD_Khatanga'] = pd.read_csv('../input/climate/real/sunshine duration/SD_Khatanga.csv')
data['SD_Chokurdakh'] = pd.read_csv('../input/climate/real/sunshine duration/SD_Chokurdakh.csv')
data['RH_Inuvik'] = pd.read_csv('../input/climate/real/relative humidity/RH_Inuvik.csv')
data['RH_Khatanga'] = pd.read_csv('../input/climate/real/relative humidity/RH_Khatanga.csv')
data['RH_Chokurdakh'] = pd.read_csv('../input/climate/real/relative humidity/RH_Chokurdakh.csv')

# Финальные вычисления со всеми данными:

In [4]:
df_COH = pd.read_excel('../input/COH/YAK_TAY_CAN_d13C_d18O_2H.xlsx')

In [15]:
#TODO: Перетащить это и ячейку ниже в .py файлы

def get_multiple_values(d: dict, keys: list) -> list:
    result = []
    existing_keys = []
    for key in keys:
        value = d.get(key)
        if not value is None:
            result.append(value)
            existing_keys.append(key)
    return existing_keys, result

In [6]:
reg_locs = {
    'TAY': 'Khatanga',
    'YAK': 'Chokurdakh',
    'CAN': 'Inuvik'
}

ind_titels = {
    'dH': '$δH$',
    'd13C': '$δ^{13}C$',
    'd18O': '$δ^{18}O$'
}

char_to_clolr = {
    'Temp': 'red',
    'Prec': 'blue',
    'VPD': 'green',
    'SD': 'orange',
    'RH': 'lightblue'
}

r_values, p_values = dict(), dict()

In [ ]:
for column in df_COH.columns:
    if column == 'Year':
        continue
    ind, reg = column.split('_')
    loc = reg_locs[reg]
    title = f'{reg_locs[reg]} {ind_titels[ind]}'
    keys, chars = get_multiple_values(data, [f'Temp_{loc}', f'Prec_{loc}', f'VPD_{loc}', f'SD_{loc}', f'RH_{loc}'])
    keys = [key.split('_')[0] for key in keys]
    colors = [char_to_clolr[key] for key in keys]

    fig, ax, rs, ps = plot_mothly_dendroclim(
        df_crn=df_COH[['Year', column]],
        dfs_char=chars,
        ylabels=keys,
        colors=colors,
        title=title,
        ylim=[-0.7, 0.8]
    )
    r_values[column] = rs
    p_values[column] = ps


In [ ]:
total_keys = ['Temp', 'Prec', 'VPD', 'SD', 'RH']

In [42]:
df_COH_corr = {
    'Month' : {'':  ['S', 'O', 'N', 'D', 'J', 'F', 'M', 'A', 'M ', 'J', 'J', 'A']},
}

In [56]:
column

'd18O_CAN'

In [57]:
for column in df_COH.columns:
    if column == 'Year':
        continue
    df_COH_corr[column] = {'Temp':[], 'Prec':[], 'VPD':[], 'RH': []}

    if not 'CAN' in column:
        # Для Инувика нет данных по Солнечному Сиянию
        df_COH_corr[column]['SD'] = []

    rs = r_values[column]
    ps = p_values[column]
    for key in rs:
        for r,p in zip(rs[key], ps[key]):
            text = f'{r:.2f}\n(p={p:.3f})'
            df_COH_corr[column][key].append(text)

df_reform = {(outerKey, innerKey): values for outerKey, innerDict in df_COH_corr.items() for innerKey, values in innerDict.items()}
df_reform = pd.DataFrame(df_reform)

In [59]:
df_reform.to_excel('../output/dendroclim_COH_corr.xlsx')

# Task 03.04.2022

Рассчитать климатику между данными в файле "TAY_YAK_COH_030422.xlsx" с темп, осадк, относит влажностью, VPD, SD c 1901-2020, и с 1950-2020 для каждого параметра С, H, О для YAK с метеост Чокурдах и Депутатское, для TAY c Хатангой. Посчитать корреляции между С, O, H для TAY и YAK для 1900-2020 and 1950-2020.

если нет данных с 50х, а только с 1966, то сделать с 1966 по 2020/2021 и до с 1900 по 1965 или с 1901 по 1950

In [4]:
df_COH = pd.read_excel("../input/COH/TAY_YAK_COH_030422.xlsx")

In [6]:
df_COH_before_1950 = df_COH[df_COH['Year']<=1950]
df_COH_after_1950 = df_COH[df_COH['Year']>1950]

In [11]:
get_df_corr(df_COH.drop(columns=['Year'])).to_excel('../output/COH_corr_1900-2021.xlsx', index=True)
get_df_corr(df_COH_before_1950.drop(columns=['Year'])).to_excel('../output/COH_corr_1900-1950.xlsx', index=True)
get_df_corr(df_COH_after_1950.drop(columns=['Year'])).to_excel('../output/COH_corr_1951-2021.xlsx', index=True)

In [7]:
loc_to_reg = {
    'Chokurdakh': 'YAK',
    'Deputatsky': 'YAK',
    'Khatanga' : 'TAY'
    }
dfs = {
    '1900-2021':df_COH,
    #'1900-1950':df_COH_before_1950,
    '1951-2021': df_COH_after_1950
    }

ind_titels = {
    '2H': '$δH$',
    '13C': '$δ^{13}C$',
    '18O': '$δ^{18}O$'
}

char_to_clolr = {
    'Temp': 'red',
    'Prec': 'blue',
    'VPD': 'green',
    'SD': 'orange',
    'RH': 'lightblue'
}



In [ ]:
for df_key in dfs:
    df_reform = get_coh_corr_table(data, dfs[df_key], loc_to_reg, ind_titels, char_to_clolr)
    df_reform.to_excel(f'../output/dendroclim_COH_corr_{df_key}.xlsx')

# Task 20.04.2022

In [14]:
df_COH = pd.read_excel("../input/COH/13C_allsites.xlsx")

In [15]:
df_sites = pd.read_csv('../input/Sites.csv')

In [16]:
regs_locs = df_sites[['Site code', 'Station name']].dropna()

In [6]:
regs = list(regs_locs['Site code'])
locs = list(regs_locs['Station name'])

In [7]:
path = '../input/climate/real'

characterisitict = [
    'precipitation',
    'relative humidity',
    'sunshine duration',
    'temperature',
    'vpd'
]


In [8]:
data = dict()

for char in characterisitict:
    for file in listdir(f'{path}/{char}'):
        data[file.split('.')[0]] = pd.read_csv(f'{path}/{char}/{file}')

In [26]:
chars = ['Temp', 'Prec', 'VPD', 'RH', 'SD']

In [27]:
periods = {
    'Station name' :locs
}

In [28]:
for char in chars:
    periods[char] = []
    for loc in locs:
        try:
            d = data[f'{char}_{loc}']
            period = f"{min(d['Year'])}-{max(d['Year'])}"
        except:
            period = ''
        periods[char] += [period]

In [30]:
pd.DataFrame(periods).to_excel('../output/Periods-table.xlsx', index=False)

In [ ]:
new_df = pd.DataFrame({'Year':range(1901,2022)})

for key in data:
    data[key]['MEAN'] = data[key].drop(columns=['Year']).mean(axis=1, skipna=True)
    new_df = new_df.merge(data[key][['Year', 'MEAN']], on='Year', how='left').rename(columns={'MEAN':key})

new_new_df = ~new_df.isna()
new_new_df['Year'] = new_df['Year']
new_new_df = new_new_df.replace(False, 0).replace(True, 1)

new_new_df.to_excel('../output/NEW_data_gap_table.xlsx')

In [9]:
dfs = {
    '1900-2021':df_COH,
    #'1951-2021': df_COH_after_1950
    }

ind_titels = {
    '2H': '$δH$',
    '13C': '$δ^{13}C$',
    '18O': '$δ^{18}O$'
}

char_to_color = {
    'Temp': 'red',
    'Prec': 'blue',
    'VPD': 'green',
    'SD': 'orange',
    'RH': 'lightblue'
}

In [17]:
df_reform = get_coh_corr_table(data, df_COH, locs, regs, ind_titels, char_to_color)

In [11]:
def highlight_cells(x):
    try:
        r, p = x.split('=')
    except:
        return None
    r = float(r[:-3])
    p = float(p[:-1])
    if p<0.05:
        if r>0:
            return 'background-color: lightgreen'
        else:
            return 'background-color: lightcoral'
    return None


In [18]:
df_reform.T.style.applymap(highlight_cells).to_excel(f'../output/dendroclim_COH_corr_FULL_03.xlsx')